In [24]:
import numpy as np

import torch
import torch.nn as nn


# Define the path to the embeddings .npy file
embeddings_npy_path = 'data/embeddings.npy'

# Load the embeddings from the .npy file
title_embeddings = np.load(embeddings_npy_path)

In [25]:
voting_dict = {
    'Y': 0,
    'N': 1,
    'A': 2,
    'X': 3,
}

In [26]:
import pandas as pd
import json

voting_data_path = 'data/UN_DATA.csv'

# read in the data
df = pd.read_csv(voting_data_path)
df['index'] = df.index

print(df)

               Council        Date  \
0     Security Council  2016-12-13   
1     Security Council  2020-06-22   
2     Security Council  2016-12-14   
3     Security Council  2016-12-12   
4     Security Council  2005-09-23   
...                ...         ...   
7850  General Assembly  1976-10-21   
7851  General Assembly  2021-12-06   
7852  General Assembly  2021-12-06   
7853  General Assembly  2021-12-06   
7854  General Assembly  2021-12-06   

                                                  Title        Resolution  \
0     Security Council resolution 2323 (2016) [on ex...  S/RES/2323(2016)   
1     Security Council resolution 2527 (2020) [on ex...  S/RES/2527(2020)   
2     Security Council resolution 2324 (2016) [on Se...  S/RES/2324(2016)   
3     Security Council resolution 2322 (2016) [on in...  S/RES/2322(2016)   
4     Security Council resolution 1627 (2005) [on ex...  S/RES/1627(2005)   
...                                                 ...               ...   
7850

<ipython-input-26-6be2abe57bd6>:7: DtypeWarning: Columns (228,229,230,232,235,236,238,239,240,241,242,243,245,246,247,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(voting_data_path)


In [31]:
title_line = df.columns.values.tolist()

print(title_line)

voters = title_line[11: -1]

voter_embedding_dim = 64

['Council', 'Date', 'Title', 'Resolution', 'TOTAL VOTES', 'NO-VOTE COUNT', 'ABSENT COUNT', 'NO COUNT', 'YES COUNT', 'Link', 'token', 'SENEGAL', 'MALAYSIA', 'VENEZUELA (BOLIVARIAN REPUBLIC OF)', 'UNITED KINGDOM', 'RUSSIAN FEDERATION', 'JAPAN', 'ANGOLA', 'URUGUAY', 'SPAIN', 'EGYPT', 'NEW ZEALAND', 'UNITED STATES', 'FRANCE', 'CHINA', 'UKRAINE', 'NIGER', 'SAINT VINCENT AND THE GRENADINES', 'ESTONIA', 'SOUTH AFRICA', 'VIET NAM', 'BELGIUM', 'GERMANY', 'TUNISIA', 'INDONESIA', 'DOMINICAN REPUBLIC', 'BRAZIL', 'UNITED REPUBLIC OF TANZANIA', 'PHILIPPINES', 'ALGERIA', 'BENIN', 'DENMARK', 'ROMANIA', 'ARGENTINA', 'GREECE', 'AUSTRIA', 'SWITZERLAND', 'CYPRUS', 'MOZAMBIQUE', 'SUDAN', 'FIJI', 'BULGARIA', 'CUBA', 'UGANDA', 'ZAMBIA', 'SOUTH SUDAN', 'MONACO', 'REPUBLIC OF KOREA', 'REPUBLIC OF MOLDOVA', 'SAN MARINO', 'YEMEN', 'ITALY', 'BURUNDI', 'HUNGARY', 'CAMBODIA', 'MALAWI', 'NAURU', 'NICARAGUA', 'BRUNEI DARUSSALAM', 'POLAND', 'THAILAND', 'MALDIVES', 'SIERRA LEONE', 'TURKEY', 'CABO VERDE', 'PAPUA NEW GUI

In [84]:

class BaseMLP(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(BaseMLP, self).__init__()

        self.voter_embeddings = nn.Embedding(len(voters), voter_embedding_dim)
        self.proj = nn.Linear(384, 64)

        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        self.relu3 = nn.ReLU()
        self.fc4 = nn.Linear(hidden_size, output_size)
        self.softmax = nn.Softmax(dim=1)
        


    def forward(self, x):
        idx = x[:, -1].long()
        title = x[:, :-1].float()

        voter_embed = self.voter_embeddings(idx)
        title_embed = self.proj(title)

        x = torch.cat((voter_embed, title_embed), dim=1)

        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.fc4(x)
        out = self.softmax(x)
        return out

In [78]:
import pandas as pd
import torch
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

class VoterDataset(Dataset):
    def __init__(self, data):
        self.data = data 
        print('data shape: ', self.data.shape)
        print(data.index)

        self.features = []
        self.labels = []

        # for each line
        for i in tqdm(range(len(self.data))):
            index = self.data.iloc[i]['index']
            cur_line = self.data.iloc[i]
            title_feature = title_embeddings[index]

            for i, voter in enumerate(voters):
                vote = cur_line[voter]

                if vote not in voting_dict:
                    continue

                feature = np.concatenate((title_feature, [i]), 0)

                self.features.append(feature)
                self.labels.append(voting_dict[vote])

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]
        

In [79]:
train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)
valid_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

train_dataset = VoterDataset(train_data)
valid_dataset = VoterDataset(valid_data)
test_dataset = VoterDataset(test_data)

data shape:  (6284, 271)
Index([6072,  432, 6001, 6308, 2118, 3832, 4398, 1108, 5523, 7678,
       ...
       6265, 5734, 3092, 3772, 5191, 5226, 5390,  860, 7603, 7270],
      dtype='int64', length=6284)


100%|██████████| 6284/6284 [00:45<00:00, 137.54it/s]


data shape:  (785, 271)
Index([6321, 1103, 2897, 1869, 1487,  828, 6353, 4477,  251, 1671,
       ...
       7658, 7847, 4999, 1172, 1242, 7204, 6607, 5260, 3859, 3121],
      dtype='int64', length=785)


100%|██████████| 785/785 [00:03<00:00, 203.53it/s]


data shape:  (786, 271)
Index([4231,  308, 1017, 6682, 6043, 1406,  334, 3194, 4270,  381,
       ...
        425, 5498, 3855, 7836, 5381, 4755, 1407, 4575,  982, 3413],
      dtype='int64', length=786)


100%|██████████| 786/786 [00:03<00:00, 256.57it/s]


In [80]:

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=64)
test_loader = DataLoader(test_dataset, batch_size=64)

In [91]:
from sklearn.metrics import f1_score

model = BaseMLP(128, 256, 4)

class_weights = torch.tensor([1, 5, 5, 5])
class_weights = class_weights / class_weights.sum()

criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
optimizer.zero_grad()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

epochs = 10
valid_loss_min = np.Inf

for epoch in range(epochs):
    train_loss = 0.0
    valid_loss = 0.0

    model.train()
    cnt = 0
    correct = 0
    total = 0
    cur_loss = 0
    for data, target in tqdm(train_loader):
        data, target = data.detach().to(device), target.to(device)
        optimizer.zero_grad()
 
        output = model(data)
        loss = criterion(output, target)
        loss.backward(retain_graph=True)
        optimizer.step()

        train_loss += loss.item() * data.size(0)
        cur_loss += loss.item()

        _ , predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()    

        cnt += 1
        if cnt % 1000 == 0:
            print('Step {} \tTraining Loss: {:.6f} \tAccuracy: {:.6f}'.format(cnt, cur_loss / 1000, correct / total))
            cur_loss = 0
            total = 0
            correct = 0

    model.eval()
    correct = 0
    total = 0
    all_predicted = []
    all_target = []
    for data, target in tqdm(valid_loader):
        data, target = data.detach().to(device), target.to(device)

        output = model(data)
        loss = criterion(output, target)

        valid_loss += loss.item() * data.size(0)

        _, predicted = torch.max(output.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

        all_predicted.extend(predicted.tolist())
        all_target.extend(target.tolist())

    train_loss = train_loss / len(train_loader.dataset)
    valid_loss = valid_loss / len(valid_loader.dataset)

    accuracy = correct / total
    f1 = f1_score(all_target, all_predicted, average='weighted')
    f1_prime = f1_score(all_target, np.zeros_like(all_predicted), average='weighted')

    print('Epoch: {} \tTraining Loss: {:.6f} \tValidation Loss: {:.6f}'.format(epoch, train_loss, valid_loss))
    print('Accuracy: {:.6f} \tF1: {:.6f} \tF1 Prime: {:.6f}'.format(accuracy, f1, f1_prime))

    if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}). Saving model ...'.format(valid_loss_min, valid_loss))
        torch.save(model.state_dict(), 'model.pt')
        valid_loss_min = valid_loss


  9%|▉         | 1019/11161 [00:08<01:09, 146.24it/s]

Step 1000 	Training Loss: 1.216822 	Accuracy: 0.655297


 18%|█▊        | 2015/11161 [00:17<01:06, 138.27it/s]

Step 2000 	Training Loss: 1.173457 	Accuracy: 0.674922


 27%|██▋       | 3027/11161 [00:24<00:55, 146.74it/s]

...

Step 10000 	Training Loss: 1.080978 	Accuracy: 0.709828


 99%|█████████▉| 11022/11161 [01:41<00:01, 121.48it/s]

Step 11000 	Training Loss: 1.082874 	Accuracy: 0.707453


100%|██████████| 1422/1422 [00:04<00:00, 303.47it/s]


Epoch: 9 	Training Loss: 1.082829 	Validation Loss: 1.082159
Accuracy: 0.725051 	F1: 0.744888 	F1 Prime: 0.659354


In [96]:
# test
model.load_state_dict(torch.load('model.pt'))

model.eval()
correct = 0
total = 0
all_predicted = []
all_target = []
for data, target in tqdm(test_loader):
    data, target = data.detach().to(device), target.to(device)

    output = model(data)

    _, predicted = torch.max(output.data, 1)
    total += target.size(0)
    correct += (predicted == target).sum().item()

    all_predicted.extend(predicted.tolist())
    all_target.extend(target.tolist())

print('Accuracy on test set: %d %%' % (100 * correct / total))
print('F1 on test set: %f' % f1_score(all_target, all_predicted, average='weighted'))
print('F1 prime on test set: %f' % f1_score(all_target, np.zeros_like(all_predicted), average='weighted'))

# try random guessing
all_predicted_random = np.random.randint(0, 4, len(all_target))
print('F1 random: %f' % f1_score(all_target, all_predicted_random, average='weighted'))


100%|██████████| 1445/1445 [00:02<00:00, 497.56it/s]


Accuracy on test set: 71 %
F1 on test set: 0.736569
F1 prime on test set: 0.654929
F1 random: 0.315826
